In [1]:
import numpy as np
from modules import bayesian
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

In [3]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesian.generatesorteddata(data, 500)

[0. 0. 1.] [0. 1. 0.] [1. 0. 0.]


In [4]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e 
#ritorno la lista dei nuovi dati 'simmetrizzati' in gplot e sd
C=bayesian.cubicarray(list(grid), True)
dic, dics=bayesian.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot1, datasigmaplot1=bayesian.datiplot(C, dic, dics)
#gplot=grid
#dataplot1=sdata[0]
#datasigmaplot1=sdata[1]
dataplot=np.log(dataplot1)
datasigmaplot=datasigmaplot1*np.sqrt(10)/dataplot1
sd=np.array([dataplot, datasigmaplot/max(datasigmaplot)])


numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [5]:
#Calcolo il fit bayesiano con glot e sd come test
N=118
mN, SN, y_infer, sy_infer, conta=\
bayesian.bayesianpol(gplot, sd, 8, N, 0.0001, gplot,bethapar=10000, ifprint= True, ifwarning= True, nLbp=0)

parametri ottimali [  -0.39053855  -29.19662418   92.09790486 -155.54012847  144.08711227
  -74.97511299   20.41969626   -2.25826676]
numero di armoniche cubiche 8
valore a kmin inferito -0.5319684019260423 bias -0.3905385472732623 dato a kmin -0.33587062547763863
determinante matrice delle armoniche cubiche ridotte: 1.5656815630810402e+36


In [6]:
#Plotto il fit e i dati (gplot e sd)
%matplotlib widget
plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer),y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [7]:
#Calcolo il best fit su gplot e sd con 50 dati
N=118
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesian.bestfit(gplot, sd, N, gplot, ifbetha=True, ifprintbestfit= False, ifprintfinal= True, nLbf=0)
#Plotto il best fit e i dati (gplot e sd)



%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

parametri ottimali [-3.18587305e-01 -3.57077357e+01  1.37068788e+02 -2.90709269e+02
  3.55553171e+02 -2.60070714e+02  1.11637089e+02 -2.58714840e+01
  2.49429092e+00]
numero di armoniche cubiche 9
valore a kmin inferito -0.4894435245663657 bias -0.31858730507743793 dato a kmin -0.33587062547763863
determinante matrice delle armoniche cubiche ridotte: 2.608968992310223e+20
grado ottimale 18 grado massimo tentato 28
numero di polinomi nella base ottimale:  9 numero di dati 118
best alpha 2.8326514391345833e-05
best betha 93.61094967750961


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [8]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(range(2,(len(log_evidence_vP)+1)*2, 2), log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#Calcolo il best fit includendo sempre piu' punti 
predbl=[]
spredbl=[]
Npointsbl=[]
N_max=101
N_iter=1
N_min=21
for i in range(2):
    predb=[]
    spredb=[]
    Npointsb=[]
    for N in range(N_min, N_max, N_iter):
        
        mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
        bayesian.bestfit(gplot, sd, N, gplot, ifbetha=True, ifprintbestfit=False, ifprintfinal=False, nLbf=i)
        Npointsb.append(np.linalg.norm(gplot[N]))
        if (N-N_min)%(((N_max-N_min))//5)==0: print(int((N-N_min)/((N_max-N_min))*100), '%')
        predb.append(mN[0])
        spredb.append(SN[0,0])
    predbl.append(predb)
    spredbl.append(spredb)
    Npointsbl.append(Npointsb)

0 %
20 %
40 %
60 %
80 %
0 %
20 %
40 %
60 %
80 %


In [10]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')
L=[0,[0,4],[0,4,6], [0,4,6,8], [0,4,6,8,10]]
for i in range(1):
    #ax[0].errorbar(np.array(Npointsbl[i]), np.array(predbl[i]), np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
    ax[0].errorbar(np.array(Npointsbl[i]), np.exp(np.array(predbl[i])), np.exp(np.array(predbl[i]))*np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
ax[0].fill_between(np.array(Npointsbl[0]), np.ones(len(Npointsbl[0]))*0.816-np.ones(len(Npointsbl[0]))*0.021, np.ones(len(Npointsbl[1]))*0.816+np.ones(len(Npointsbl[1]))*0.021, label='GK result', alpha=0.3)    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot1,datasigmaplot1*np.sqrt(10.), fmt='.', label='3375 water molecules at 300k 30ns') 
ax[1].fill_between(np.linalg.norm(gplot, axis=1), np.ones(len(gplot))*0.816-np.ones(len(gplot))*0.021, np.ones(len(gplot))*0.816+np.ones(len(gplot))*0.021, label='GK result', alpha=0.3)    
ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'k ($nm^{-1}$)')

In [26]:
datimg=np.loadtxt('Dkkmax.text')

In [52]:
datimg[0]
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')
L=[0,[0,4],[0,4,6], [0,4,6,8], [0,4,6,8,10]]
for i in range(1):
    #ax[0].errorbar(np.array(Npointsbl[i]), np.array(predbl[i]), np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
    ax[0].errorbar(np.array(Npointsbl[i]), np.exp(np.array(predbl[i])), np.exp(np.array(predbl[i]))*np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
ax[0].fill_between(np.array(Npointsbl[0]), np.ones(len(Npointsbl[0]))*0.816-np.ones(len(Npointsbl[0]))*0.021, np.ones(len(Npointsbl[1]))*0.816+np.ones(len(Npointsbl[1]))*0.021, label='GK result', alpha=0.3)
ax[0].errorbar(datimg[0]*10, datimg[1], datimg[2], fmt='.', label='mg')
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot1,datasigmaplot1*np.sqrt(10.), fmt='.', label='3375 water molecules at 300k 30ns') 
ax[1].fill_between(np.linalg.norm(gplot, axis=1), np.ones(len(gplot))*0.816-np.ones(len(gplot))*0.021, np.ones(len(gplot))*0.816+np.ones(len(gplot))*0.021, label='GK result', alpha=0.3)    
ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'k ($nm^{-1}$)')